In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
.appName('GroupByKey vs ReduceByKey').getOrCreate()

25/06/01 03:16:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
!hadoop fs -ls /tmp/

Found 10 items
-rw-r--r--   2 geurecruitanandraj hadoop    1060750 2025-05-14 10:17 /tmp/customers.csv
-rw-r--r--   2 geurecruitanandraj hadoop   10528211 2025-05-14 10:27 /tmp/customers_10mb.csv
-rw-r--r--   2 geurecruitanandraj hadoop  168541068 2025-05-14 12:27 /tmp/customers_150mb.csv
drwxrwxrwt   - hdfs               hadoop          0 2025-04-26 14:44 /tmp/hadoop-yarn
drwx-wx-wx   - hive               hadoop          0 2025-04-26 14:44 /tmp/hive
-rw-r--r--   2 root               hadoop         78 2025-05-01 14:03 /tmp/inputhdfsbdz.txt
-rw-r--r--   2 geurecruitanandraj hadoop    8527842 2025-05-17 14:51 /tmp/items_10.csv
-rw-r--r--   2 geurecruitanandraj hadoop    8628599 2025-05-17 14:51 /tmp/orders_10.csv
-rw-r--r--   2 geurecruitanandraj hadoop    8542328 2025-05-17 14:58 /tmp/payments_10.csv
-rw-r--r--   2 geurecruitanandraj hadoop    7896035 2025-05-17 14:59 /tmp/shippings_10.csv


In [4]:
hdfs_path = '/tmp/customers.csv'
rdd = spark.sparkContext.textFile(hdfs_path)

In [5]:
header = rdd.first()


In [7]:
rdd_no_header = rdd.filter(lambda row:row!=header).map(lambda row:row.split(','))

In [8]:
rdd_no_header.first()

['0', 'Customer_0', 'Pune', 'Maharashtra', 'India', '2023-06-29', 'False']

In [9]:
reduced_rdd = rdd_no_header.map(lambda row:(row[2],1)).reduceByKey(lambda x,y:x+y)

In [10]:
reduced_rdd.collect()

[('Pune', 2243),
 ('Hyderabad', 2242),
 ('Mumbai', 2142),
 ('Delhi', 2200),
 ('Bangalore', 2211),
 ('Ahmedabad', 2198),
 ('Chennai', 2194),
 ('Kolkata', 2223)]

In [11]:
grouped_rdd = rdd_no_header.map(lambda row:(row[2],1)).groupByKey()

In [12]:
grouped_rdd.collect()

[('Pune', <pyspark.resultiterable.ResultIterable at 0x7f257ed2d1d0>),
 ('Hyderabad', <pyspark.resultiterable.ResultIterable at 0x7f257eea5250>),
 ('Mumbai', <pyspark.resultiterable.ResultIterable at 0x7f257ed2dad0>),
 ('Delhi', <pyspark.resultiterable.ResultIterable at 0x7f258c05d5d0>),
 ('Bangalore', <pyspark.resultiterable.ResultIterable at 0x7f257ed2e3d0>),
 ('Ahmedabad', <pyspark.resultiterable.ResultIterable at 0x7f257ffbdbd0>),
 ('Chennai', <pyspark.resultiterable.ResultIterable at 0x7f257ed2e9d0>),
 ('Kolkata', <pyspark.resultiterable.ResultIterable at 0x7f257ed2dfd0>)]

In [13]:
grouped_rdd_result = grouped_rdd.map(lambda row:(row[0],len(row[1])))

In [14]:
grouped_rdd_result.collect()

[('Pune', 2243),
 ('Hyderabad', 2242),
 ('Mumbai', 2142),
 ('Delhi', 2200),
 ('Bangalore', 2211),
 ('Ahmedabad', 2198),
 ('Chennai', 2194),
 ('Kolkata', 2223)]

## ReduceBykey cannot be used on non-commulative and non-associative property like mean ,median , we could be used in commullative data like min,max, sum etc

In [15]:
spark.stop()